# Testing for Ghosts in the Machine: Assuring 'Good Enough' Software Quality in AI-based systems<br>

## Artur Patoka
PyCon Italia<br>
Florence 2024

![prace_logos.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/prace_logos.png)

# Problem statement

![1.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/1.png)

![2.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/2.png)

![3.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/3.png)

![4.GIF](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/4.GIF)

## Display a cell with its output

In [ ]:
import sme

model = sme.open_example_model()
print(model)

## Output can be plots or images

![screen_empire.png](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/screen_empire.png)

![screen_empire.GIF](https://raw.githubusercontent.com/arturpat/pycon-it-pres-2/main/screen_empire.GIF)

## Or even videos

In [ ]:
import sme_contrib.plot as smeplot
from IPython.display import HTML

results = model.simulate(100, 10)
anim = smeplot.concentration_heatmap_animation(results, ["B_cell"])
HTML(anim.to_html5_video())

## More information

- https://medium.com/@mjspeck/presenting-code-using-jupyter-notebook-slides-a8a3c3b59d67
- https://www.blog.pythonlibrary.org/2018/09/25/creating-presentations-with-jupyter-notebook/
- https://rise.readthedocs.io/
- https://github.com/ssciwr/jupyter-slides-template